In [ ]:
import sagas.crmsfa.odoo_facade
import imp
imp.reload(sagas.crmsfa.odoo_facade)

In [1]:
from sagas.tool.app_settings import setup_logger, setup_jupyter
setup_logger('crmsfa.log')
setup_jupyter()

In [1]:
from sagas.crmsfa.odoo_facade import odoo, login
login()

if 'sale.order' in odoo.env:
    Order = odoo.env['sale.order']
    order_ids = Order.search([])
    print(order_ids)
else:
    print("no sale.order model")

[33, 30, 28, 8, 7, 6, 4, 3, 32, 31, 29, 27, 26, 25, 24, 23, 22, 21, 14, 9, 15, 10, 16, 11, 17, 12, 34, 20, 19, 5, 2, 1, 35, 18, 13, 36]


In [2]:
user = odoo.env.user
print(user.name)            # name of the user connected
print(user.company_id.name) # the name of its company

samlet@sagas.ai
YourCompany


In [3]:
Country = odoo.env['res.country']
de_id = Country.search([('code', '=', 'DE')])[0]
de = Country.browse(de_id)
print(de)
print(de.name)

Recordset('res.country', [57])
Germany


In [11]:
rs_ids=Country.search([])
rs=Country.browse(rs_ids)
for el in rs:
    print(el.code, el.name)

AF Afghanistan
AL Albania
DZ Algeria
AS American Samoa
AD Andorra
AO Angola
AI Anguilla
AQ Antarctica
AG Antigua and Barbuda
AR Argentina
AM Armenia
AW Aruba
AU Australia
AT Austria
AZ Azerbaijan
BS Bahamas
BH Bahrain
BD Bangladesh
BB Barbados
BY Belarus
BE Belgium
BZ Belize
BJ Benin
BM Bermuda
BT Bhutan
BO Bolivia
BQ Bonaire, Sint Eustatius and Saba
BA Bosnia and Herzegovina
BW Botswana
BV Bouvet Island
BR Brazil
IO British Indian Ocean Territory
BN Brunei Darussalam
BG Bulgaria
BF Burkina Faso
BI Burundi
KH Cambodia
CM Cameroon
CA Canada
CV Cape Verde
KY Cayman Islands
CF Central African Republic
TD Chad
CL Chile
CN China
CX Christmas Island
CC Cocos (Keeling) Islands
CO Colombia
KM Comoros
CG Congo
CK Cook Islands
CR Costa Rica
HR Croatia
CU Cuba
CW Curaçao
CY Cyprus
CZ Czech Republic
CI Côte d'Ivoire
CD Democratic Republic of the Congo
DK Denmark
DJ Djibouti
DM Dominica
DO Dominican Republic
EC Ecuador
EG Egypt
SV El Salvador
GQ Equatorial Guinea
ER Eritrea
EE Estonia
ET Ethiopia
F

In [4]:
import odoorpc.fields as fields
from odoorpc.env import FIELDS_RESERVED
from tabulate import tabulate
import wcwidth

def desc_model(model):
    table_header = ['name','type', 'string']
    table_data = []

    attrs = {
                '_name': model,
                '_columns': {},
            }
    fields_get = odoo.execute(model, 'fields_get')
    for field_name, field_data in fields_get.items():
        if field_name not in FIELDS_RESERVED:
            Field = fields.generate_field(field_name, field_data)
            attrs['_columns'][field_name] = Field
            attrs[field_name] = Field
            # print(field_name, "%", field_data['type'])
            # print(field_data.keys())
            # ['searchable', 'sortable', 'depends', 'store', 
            # 'manual', 'type', 'company_dependent', 
            # 'change_default', 'readonly', 'string', 'required']
            table_data.append((field_name, field_data['type'], 
                               field_data['string']))
            # addi: type(Field)

    # print(tabulate(table_data, headers=table_header, tablefmt='grid'))
    # print(tabulate(table_data, headers=table_header, tablefmt='fancy_grid'))
    print(tabulate(table_data, headers=table_header, tablefmt='psql'))

model="res.country"
desc_model(model)

+-------------------+-----------+------------------------+
| name              | type      | string                 |
|-------------------+-----------+------------------------|
| write_uid         | many2one  | Last Updated by        |
| image             | binary    | Image                  |
| code              | char      | Country Code           |
| name              | char      | Country Name           |
| country_group_ids | many2many | Country Groups         |
| __last_update     | datetime  | Last Modified on       |
| address_view_id   | many2one  | Input View             |
| write_date        | datetime  | Last Updated on        |
| create_uid        | many2one  | Created by             |
| create_date       | datetime  | Created on             |
| address_format    | text      | Layout in Reports      |
| display_name      | char      | Display Name           |
| name_position     | selection | Customer Name Position |
| vat_label         | char      | Vat Label             

In [21]:
desc_model('mail.thread')

+----------------------------+-----------+-------------------------+
| name                       | type      | string                  |
|----------------------------+-----------+-------------------------|
| message_partner_ids        | many2many | Followers (Partners)    |
| message_follower_ids       | one2many  | Followers               |
| message_ids                | one2many  | Messages                |
| message_unread             | boolean   | Unread Messages         |
| website_message_ids        | one2many  | Website Messages        |
| message_needaction_counter | integer   | Number of Actions       |
| message_needaction         | boolean   | Action Needed           |
| display_name               | char      | Display Name            |
| message_is_follower        | boolean   | Is Follower             |
| message_last_post          | datetime  | Last Message Date       |
| __last_update              | datetime  | Last Modified on        |
| message_unread_counter     | int

In [22]:
desc_model('crm.lead')

+----------------------------+-----------+-------------------------+
| name                       | type      | string                  |
|----------------------------+-----------+-------------------------|
| message_partner_ids        | many2many | Followers (Partners)    |
| user_login                 | char      | User Login              |
| email_from                 | char      | Email                   |
| message_unread             | boolean   | Unread Messages         |
| medium_id                  | many2one  | Medium                  |
| partner_id                 | many2one  | Customer                |
| title                      | many2one  | Title                   |
| source_id                  | many2one  | Source                  |
| display_name               | char      | Display Name            |
| referred                   | char      | Referred By             |
| name                       | char      | Opportunity             |
| user_id                    | man

In [5]:
model="res.partner"
model_exists = odoo.execute('ir.model', 'search',
                                          [('model', '=', model)])
print(bool(model_exists))

True


In [7]:
from tabulate import tabulate
import wcwidth
from sagas.crmsfa.odoo_facade import odoo, login

def model_info():
    login()

    # models = odoo.execute('ir.model', 'browse', [])
    # print(models)

    # https://stackoverflow.com/questions/2872512/python-truncate-a-long-string
    def abbrev(data, l=15):
        info = (data[:l] + '..') if len(data) > l else data
        return info

    table_header = ['model','name', 'count']
    table_data = []

    Order = odoo.env['ir.model']
    order_ids = Order.search([])
    print(len(order_ids))
    models=Order.browse(order_ids)
    # print(models[0])
    # odoo/addons/base/ir/ir_model.py
    for order in models:
        # print(order.model, order.name, order.count)    
        table_data.append((abbrev(order.model), 
                           abbrev(order.name), order.count))

    print(tabulate(table_data, headers=table_header, tablefmt='psql'))    

model_info()

360
+-------------------+----------------------------------+---------+
| model             | name                             |   count |
|-------------------+----------------------------------+---------|
| mail.thread       | EMail线程                        |       0 |
| product.product   | 产品                             |      39 |
| product.templat.. | 产品模板                         |      32 |
| account.payment   | 付款                             |       0 |
| project.task      | 任务                             |      31 |
| account.analyti.. | 分析账户                         |      18 |
| account.invoice   | 发票                             |       0 |
| stock.productio.. | 批次/序列号                      |       1 |
| calendar.event    | 活动                             |       7 |
| crm.lead          | 线索/商机                        |      44 |
| res.partner       | 联系人                           |      43 |
| mail.channel      | 讨论频道                         |       6 |
| stock.picking     | 调拨   

In [4]:
tabulate.WIDE_CHARS_MODE=True